In [ ]:
#Given raw image features, calculate wordnet domain features

In [ ]:
import pandas as pd
import numpy as np
import copy
from nltk.corpus import wordnet as wn

In [ ]:
#Change these variables to work with your data
home = '/local/image_personality/image_data/data2016/'
IMAGE_FEATURES_FILE = home+"image_features/features_image_all_averages.csv" #all average object detections
SENSEMAP_FILE_MONO = "/local/software/wn-domains-3.2/sensemap-2.1-3.0/2.1to3.0.noun.mono" #Mapping WordNet domains from WordNet 2.1 to WordNet 3.0 for monosemous nouns
SENSEMAP_FILE_POLY = "/local/software/wn-domains-3.2/sensemap-2.1-3.0/2.1to3.0.noun.poly" #Mapping WordNet domains from WordNet 2.1 to WordNet 3.0 for polysemous nouns
WORDNET_DOMAINS = "/local/software/wn-domains-3.2/wn-domains-3.2-20070223"
OUTPUT_FILE = home+"image_features/features_objects_wordnet_domains.csv"

In [ ]:
imageFeatures = pd.read_csv(IMAGE_FEATURES_FILE)
imageFeatures = imageFeatures.drop("Unnamed: 0",axis=1)

In [ ]:
objs = [i for i in imageFeatures.columns.values if i[:8]=="IMAGENET"]

In [ ]:
#Load WordNet 2.1 to 3.0 mappings
mono_mapping0 = pd.read_csv(SENSEMAP_FILE_MONO,header=None,sep=' ')

mono_mapping0['2.1_sense_key'] = [i[:-2] for i in mono_mapping0[0]]
mono_mapping0['2.1_synset_offset'] = mono_mapping0[1]
mono_mapping0['3.0_sense_key'] = [i[:-2] for i in mono_mapping0[2]]
mono_mapping0['3.0_synset_offset'] = mono_mapping0[3]
mono_mapping0 = mono_mapping0.drop([0,1,2,3],axis=1)

In [ ]:
poly_mapping0 = pd.read_csv(SENSEMAP_FILE_POLY,header=None)

poly_mapping0['score'] = [int(i.split(' ')[0]) for i in poly_mapping0[0]]
poly_mapping0['2.1_sense_info'] = [i.split(' ')[1] for i in poly_mapping0[0]]
poly_mapping0['3.0_sense_info_first'] = [i.split(' ')[2] if len(i.split(' ')) > 2 else '' for i in poly_mapping0[0]]
poly_mapping0['3.0_sense_info_rest'] = [i.split(' ')[3:] if len(i.split(' ')) > 3 else '' for i in poly_mapping0[0]]

poly_mapping0['2.1_sense_key'] = [i.split(';')[0] for i in poly_mapping0['2.1_sense_info']]
poly_mapping0['2.1_synset_offset'] = [i.split(';')[1] for i in poly_mapping0['2.1_sense_info']]
poly_mapping0['2.1_sense_number'] = [i.split(';')[2] for i in poly_mapping0['2.1_sense_info']]
poly_mapping0['2.1_sense_key'] = [i[:-2] for i in poly_mapping0['2.1_sense_key']]

poly_mapping0['3.0_first_sense_key'] = [i.split(';')[0] if len(i.split(';')) > 0 else '' for i in poly_mapping0['3.0_sense_info_first']]
poly_mapping0['3.0_first_synset_offset'] = [i.split(';')[1] if len(i.split(';')) > 1 else '' for i in poly_mapping0['3.0_sense_info_first']]
poly_mapping0['3.0_first_sense_number'] = [i.split(';')[2] if len(i.split(';')) > 2 else '' for i in poly_mapping0['3.0_sense_info_first']]
poly_mapping0['3.0_first_sense_key'] = [i[:-2] for i in poly_mapping0['3.0_first_sense_key']]

poly_mapping0 = poly_mapping0.drop([0,'2.1_sense_info','3.0_sense_info_first'],axis=1)

In [ ]:
#Load WordNet 2.0 to 2.1 mappings
mono_mapping1 = pd.read_csv("/local/software/wn-domains-3.2/sensemap-2.0-2.1/2.0to2.1.noun.mono",header=None,sep=' ')

mono_mapping1['2.0_sense_key'] = [i[:-2] for i in mono_mapping1[0]]
mono_mapping1['2.0_synset_offset'] = mono_mapping1[1]
mono_mapping1['2.1_sense_key'] = [i[:-2] for i in mono_mapping1[2]]
mono_mapping1['2.1_synset_offset'] = mono_mapping1[3]
mono_mapping1 = mono_mapping1.drop([0,1,2,3],axis=1)

In [ ]:
poly_mapping1 = pd.read_csv("/local/software/wn-domains-3.2/sensemap-2.0-2.1/2.0to2.1.noun.poly",header=None)

poly_mapping1['score'] = [int(i.split(' ')[0]) for i in poly_mapping1[0]]
poly_mapping1['2.0_sense_info'] = [i.split(' ')[1] for i in poly_mapping1[0]]
poly_mapping1['2.1_sense_info_first'] = [i.split(' ')[2] if len(i.split(' ')) > 2 else '' for i in poly_mapping1[0]]
poly_mapping1['2.1_sense_info_rest'] = [i.split(' ')[3:] if len(i.split(' ')) > 3 else '' for i in poly_mapping1[0]]

poly_mapping1['2.0_sense_key'] = [i.split(';')[0] for i in poly_mapping1['2.0_sense_info']]
poly_mapping1['2.0_synset_offset'] = [i.split(';')[1] for i in poly_mapping1['2.0_sense_info']]
poly_mapping1['2.0_sense_number'] = [i.split(';')[2] for i in poly_mapping1['2.0_sense_info']]
poly_mapping1['2.0_sense_key'] = [i[:-2] for i in poly_mapping1['2.0_sense_key']]

poly_mapping1['2.1_first_sense_key'] = [i.split(';')[0] if len(i.split(';')) > 0 else '' for i in poly_mapping1['2.1_sense_info_first']]
poly_mapping1['2.1_first_synset_offset'] = [i.split(';')[1] if len(i.split(';')) > 1 else '' for i in poly_mapping1['2.1_sense_info_first']]
poly_mapping1['2.1_first_sense_number'] = [i.split(';')[2] if len(i.split(';')) > 2 else '' for i in poly_mapping1['2.1_sense_info_first']]
poly_mapping1['2.1_first_sense_key'] = [i[:-2] for i in poly_mapping1['2.1_first_sense_key']]

poly_mapping1 = poly_mapping1.drop([0,'2.0_sense_info','2.1_sense_info_first'],axis=1)

In [ ]:
#Load WordNet Domains
wnDomains = pd.read_csv(WORDNET_DOMAINS,header=None,sep='\t')

In [ ]:
domains = []

for obj in objs:    
    #Map from WordNet 3.0 to WordNet 2.1
    offset_3 = obj.split(' ')[1][1:]
    if len(mono_mapping0.loc[mono_mapping0['3.0_synset_offset']==int(offset_3)]) > 0: #found in mono_mapping
        offset_21 = str(mono_mapping0.loc[mono_mapping0['3.0_synset_offset']==int(offset_3)].iloc[0]['2.1_synset_offset'])
        while len(offset_21) < 8:
            offset_21 = '0' + offset_21
    elif len(poly_mapping0.loc[poly_mapping0['3.0_first_synset_offset']==offset_3]) > 0: #found in poly_mapping
        offset_21 = poly_mapping0.loc[poly_mapping0['3.0_first_synset_offset']==offset_3].sort_values(by='score',ascending=False).iloc[0]['2.1_synset_offset']
    else: #not found in either
        domains.append('')
        continue
        
    #print('offset_3',offset_3)
    #print('offset_21',offset_21)
        
    #Map from WordNet 2.1 to WordNet 2.0
    if len(mono_mapping1.loc[mono_mapping1['2.1_synset_offset']==int(offset_21)]) > 0: #found in mono_mapping
        offset_2 = str(mono_mapping1.loc[mono_mapping1['2.1_synset_offset']==int(offset_21)].iloc[0]['2.0_synset_offset'])
        while len(offset_2) < 8:
            offset_2 = '0' + offset_2
    elif len(poly_mapping1.loc[poly_mapping1['2.1_first_synset_offset']==offset_21]) > 0: #found in poly_mapping
        offset_2 = poly_mapping1.loc[poly_mapping1['2.1_first_synset_offset']==offset_21].sort_values(by='score',ascending=False).iloc[0]['2.0_synset_offset']
    else: #not found in either
        domains.append('')
        continue
        
    #print('offset_2',offset_2)
        
    #Figure out WordNet Domains
    if len(wnDomains.loc[wnDomains[0]==offset_2+'-n']) > 0:
        domains.append(wnDomains.loc[wnDomains[0]==offset_2+'-n'].iloc[0][1])
    else:
        domains.append('')

In [ ]:
final_mapping = pd.DataFrame()
final_mapping['object'] = objs
final_mapping['domains'] = domains

In [ ]:
final_mapping.loc[final_mapping['domains']==""]

In [ ]:
domains = {}
for it,row in final_mapping.iterrows():
    d = row['domains'].split(' ')
    for dom in d:
        if dom in domains:
            domains[dom].append(row['object'])
        else:
            domains[dom] = [row['object']]

In [ ]:
len(domains)

In [ ]:
domainFeatures = pd.DataFrame(index=imageFeatures.index)
domainFeatures['id'] = imageFeatures['id']

In [ ]:
for domain,objects in domains.items():
    total = imageFeatures[objects[0]]
    for obj in objects[1:]:
        total += imageFeatures[obj]
    domainFeatures[domain] = total

In [ ]:
#Now, combine all the smaller domains into the basic domains (to use for correlations)
#This hierarchy comes from wn-domains-3.2/WDH-to-DDC-map.pdf
#Not all domains are here because not all domains are represented in our 1,000 object categories
basic_domains = {}
basic_domains['history'] = ['history']
basic_domains['art'] = ['art','painting','music','photography','theatre','cinema','jewellery']
basic_domains['religion'] = ['religion']
basic_domains['play'] = ['play']
basic_domains['radio+tv'] = ['radio+tv']
basic_domains['sport'] = ['sport','baseball','basketball','football','golf','rugby','soccer','table_tennis',\
                         'tennis','volleyball','skiing','hockey','swimming','racing','athletics','boxing',\
                         'fishing','hunting']
basic_domains['agriculture'] = ['agriculture','animal_husbandry']
basic_domains['food'] = ['food','gastronomy']
basic_domains['home'] = ['home']
basic_domains['architecture'] = ['architecture','town_planning','buildings','furniture']
basic_domains['computer_science'] = ['computer_science']
basic_domains['engineering'] = ['engineering','mechanics','astronautics','electrotechnology']
basic_domains['telecommunication'] = ['telecommunication','telephony']
basic_domains['medicine'] = ['medicine','pharmacy']
basic_domains['astronomy'] = ['astronomy']
basic_domains['biology'] = ['biology']
basic_domains['animals'] = ['animals','entomology']
basic_domains['plants'] = ['plants']
basic_domains['chemistry'] = ['chemistry']
basic_domains['earth'] = ['geology','metrology','geography']
basic_domains['mathematics'] = ['mathematics']
basic_domains['physics'] = ['physics','acoustics','atomic_physic','electronics','optics']
basic_domains['anthropology'] = ['folklore']
basic_domains['health'] = ['body_care']
basic_domains['military'] = ['military']
basic_domains['publishing'] = ['publishing']
basic_domains['artisanship'] = ['artisanship']
basic_domains['commerce'] = ['commerce']
basic_domains['industry'] = ['industry']
basic_domains['transport'] = ['transport','aviation','nautical','railway','vehicles']
basic_domains['economy'] = ['economy','enterprise','banking','money']
basic_domains['administration'] = ['administration']
basic_domains['law'] = ['law']
basic_domains['tourism'] = ['tourism']
basic_domains['fashion'] = ['fashion']

In [ ]:
for word in domains.keys():
    found = False
    for domain in basic_domains:
        if word in basic_domains[domain]:
            found = True
            break
    if not found:
        print(word)

In [ ]:
for domain,words in basic_domains.items():
    temp = domainFeatures[words[0]]
    for word in words[1:]:
        temp += domainFeatures[word]
    domainFeatures['basic_'+domain] = temp

In [ ]:
domainFeatures.to_csv(OUTPUT_FILE)